In [ ]:
from carbonplan_retro.data import cat

import pandas as pd

import numpy as np
import statsmodels.formula.api as smf

import seaborn as sns
from carbonplan import styles
import matplotlib.pyplot as plt

styles.mpl.set_theme(style="carbonplan_light", font_scale=1.5)
colors = styles.colors.light

In [ ]:
retro_json = cat.retro_db_light_json.read()
projects = [
    project
    for project in retro_json
    if project["carbon"]["initial_carbon_stock"]["value"]
    > project["carbon"]["common_practice"]["value"]
]

In [ ]:
def get_summary_stats(data):
    summary_d = {}
    mod = smf.ols("cp~average_slag_baseline", data=data).fit()
    summary_d["slope"] = mod.params["average_slag_baseline"]

    delta_slag = data["average_slag_baseline"] - data["cp"]
    desc = delta_slag.describe()
    summary_d["median_delta"] = desc["50%"]
    summary_d["mean_delta"] = desc["mean"]
    return summary_d

In [ ]:
df = pd.DataFrame(
    {
        project["opr_id"]: (
            project["carbon"]["average_slag_baseline"]["value"],
            project["carbon"]["common_practice"]["value"],
        )
        for project in projects
    }
).T.rename(columns={0: "average_slag_baseline", 1: "cp"})

In [ ]:
df = df.dropna()

In [ ]:
df.sort_values(by=["cp"], inplace=True, ascending=False)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 4))

# ax.bar(df.index.values, df['cp'], width=3, fc='None')
ax.scatter(
    df.index.values, df["cp"], marker="_", s=200, color=".3", lw=2.5, label="ARB Minimum Baseline"
)
ax.scatter(
    df.index.values,
    df["average_slag_baseline"],
    marker="o",
    s=40,
    color="#7EB36A",
    label="Baseline Average",
)
plt.ylim(0, 200)
plt.tick_params(axis="x", which="major", rotation=90)
ax.xaxis.set_visible(False)  # axes()['lower'].set(visible=False)
plt.xlim(-1, 65)
plt.legend(frameon=False)
ax.spines["bottom"].set_visible(False)

med_diff = np.median(df["average_slag_baseline"] - df["cp"])
mean_diff = np.mean(df["average_slag_baseline"] - df["cp"])

ax.annotate(
    f"Median $\Delta$: {med_diff:.1f}\n  Mean $\Delta$: {mean_diff:.1f}",
    xy=(0.8325, 0.51),
    xycoords="figure fraction",
)
plt.ylabel("Aboveground Carbon\n(tCO2e acre$^{-1}$)")

plt.savefig("/home/jovyan/pub-figs/convergence.eps")

## Scatter and Barplots


In [ ]:
no_impute_summary = get_summary_stats(df)

fig, (ax1) = plt.subplots(1, 1, figsize=(3.5, 3.5))
sns.regplot(
    x="average_slag_baseline",
    y="cp",
    data=df,
    ax=ax1,
    scatter_kws={"s": 165, "alpha": 0.55, "color": ".3"},
    fit_reg=False,
)
x_max = 200
ax1.plot((0, x_max), (0, x_max), lw=3, ls="--", color="r")
ax1.set_xlim(0, x_max)
ax1.set_ylim(0, x_max)

shared_ticks = np.linspace(0, x_max, 3)

ax1.set_yticks(shared_ticks)
ax1.set_xticks(shared_ticks)

slag_units = "tCO2e acre$^{-1}$"

ax1.set_xlabel(f"Baseline Average\n({slag_units})")
ax1.set_ylabel(f"Common Practice\n({slag_units})")

ax1.annotate(
    f"Slope: {str(no_impute_summary['slope'])[:4]}",
    xy=(0.55, 0.26),
    xycoords="axes fraction",
    fontsize=15,
)
ax1.annotate(
    f"Median $\Delta$: {no_impute_summary['median_delta']:.0f}",
    xy=(0.55, 0.18),
    xycoords="axes fraction",
    fontsize=15,
)
ax1.annotate(
    f"Mean $\Delta$: {no_impute_summary['mean_delta']:.2f}",
    xy=(0.55, 0.1),
    xycoords="axes fraction",
    fontsize=15,
)


sns.despine()

out_fn = "/home/jovyan/pub-figs/baseline_common_practice_scatter.pdf"
plt.savefig(out_fn, dpi=300, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))
sns.histplot(
    df["average_slag_baseline"] - df["cp"],
    binwidth=5,
    color=".3",
    ax=ax,
    alpha=0.75,
    kde=False,
    element="bars",
    fill=True,
    # line_kws=dict(edgecolor="k", linewidth=0),
)

ax.set_xticks(np.linspace(0, 30, 3))

ax.set_xlabel(f"Baseline Carbon minus\nCommon Practice\n({slag_units})")

sns.despine()

out_fn = "/home/jovyan/pub-figs/baseline_common_practice_hist.pdf"
plt.savefig(out_fn, dpi=300, bbox_inches="tight")